Script used in case study of my Master's thesis utilizing the GitHub and Debricked API.
Please read readme for more details and how to use.

In [ ]:
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pickle5
import requests
import base64
import json 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta
import os
import pickle5 as pickle

In [ ]:
repos = [{
    'identifier': '',
    'dependency_files': [],
}]

In [ ]:
github_api_token = ''
base_repos_url = 'https://api.github.com/repos/'
debricked_access_token = ''
debricked_api_version = '1.0'
debricked_api_url = f'https://debricked.com/api/{debricked_api_version}'

headers = {}
headers['Authorization'] = f"token {github_api_token}"

In [ ]:
"""
    Created by Christian Normand
    Copied from https://medium.com/@ctnormand1/the-four-quadrant-chart-3debe0dfdd77 
    With custom modifications to improve readability
"""
def quadrant_chart(x, y, xtick_labels=None, ytick_labels=None, data_labels=None,
 highlight_quadrants=None, ax=None):
    """
    Create the classic four-quadrant chart.
    Args:
        x -- array-like, the x-coordinates to plot
        y -- array-like, the y-coordinates to plot
        xtick_labels -- list, default: None, a two-value list xtick labels
        ytick_labels -- list, default: None, a two-value list of ytick labels
        data_labels -- array-like, default: None, data point annotations
        highlight_quadrants -- list, default: None, list of quadrants to
            emphasize (quadrants are numbered 1-4)
        ax -- matplotlib.axes object, default: None, the user can pass their own
            axes object if desired
    """
    # allow user to specify their own axes
    ax = ax if ax else plt.axes()

    data = pd.DataFrame({'x': x, 'y': y, 'data_labels': data_labels})

    # calculate averages up front to avoid repeated calculations
    y_avg = data['y'].mean()
    x_avg = data['x'].mean()

    # set x limits
    adj_x = max((data['x'].max() - x_avg), (x_avg - data['x'].min())) * 1.1
    lb_x, ub_x = (x_avg - adj_x, x_avg + adj_x)
    ax.set_xlim(lb_x, ub_x)

    # set y limits
    adj_y = max((data['y'].max() - y_avg), (y_avg - data['y'].min())) * 1.1
    lb_y, ub_y = (y_avg - adj_y, y_avg + adj_y)
    ax.set_ylim(lb_y, ub_y)

    # set x tick labels
    if xtick_labels:
        ax.set_xticks([(x_avg - adj_x / 2), (x_avg + adj_x / 2)])
        ax.set_xticklabels(xtick_labels)

    # set y tick labels
    if ytick_labels:
        ax.set_yticks([(y_avg - adj_y / 2), (y_avg + adj_y / 2)])
        ax.set_yticklabels(ytick_labels, rotation='vertical', va='center')

    # determine which points to highlight
    if highlight_quadrants:
        quadrants = []
        for x_val, y_val in zip(x, y):
            q = []
            if (x_val >= x_avg) and (y_val >= y_avg):
                q.append(1)
            if (x_val <= x_avg) and (y_val >= y_avg):
                q.append(2)
            if (x_val <= x_avg) and (y_val <= y_avg):
                q.append(3)
            if (x_val >= x_avg) and (y_val <= y_avg):
                q.append(4)
            quadrants.append(q)
        data['quadrant'] = quadrants

        # boolean mask - True = highlight, False = don't highlight
        highlight = data['quadrant'].apply(lambda q: len(set(
        highlight_quadrants) & set(q)) > 0)

        # plot the non-highlighted points within the conditional block
        ax.scatter(data['x'][~highlight], data['y'][~highlight], alpha=0.5,
        c='lightblue', edgecolor='darkblue', zorder=99)
        data = data[highlight]

    # plot remaining points and quadrant lines
    ax.scatter(x=data['x'], y=data['y'], c='lightblue', edgecolor='darkblue',
    zorder=99)
    ax.axvline(x_avg, c='k', lw=1)
    ax.axhline(y_avg, c='k', lw=1)

    # add data labels
    for ix, row in data.iterrows():
        labels = ax.annotate(row['data_labels'], (row['x'], row['y']), xytext=(2, [-38,18][random.randrange(2)]),
        textcoords='offset pixels', fontsize=11)

In [ ]:
def get_repo_language_data(repos: list, overwrite_existing_data = False) -> list:
    # prepare for data export
    file_name_repo_language_data = "all_repo_language_data.pkl"
    existing_file_found = False
    try:
        f = open(file_name_repo_language_data, "rb")
        existing_file_found = True
    except:
        existing_file_found = False
    if (existing_file_found and not overwrite_existing_data):
        f = open(file_name_repo_language_data, "rb")
        return pickle.load(f)
    else:
        all_languages = []
        for repo in repos:
            res = requests.get(f"{base_repos_url}{repo['identifier']}/languages" , headers=headers )
            data = res.json()
            sum_byte = sum(data.values())
            lang_share = {k: round((v / sum_byte), 6 ) for k, v in data.items()}
            all_languages += lang_share.items()
        # dump data to file
        open_file = open(file_name_repo_language_data, "wb")
        pickle.dump(all_languages, open_file)
        open_file.close()
        return all_languages

def get_language_apperance(language_data: list):
    sum_lang_apperances = {k: sum(x[0] == k for x in language_data) for k, v in language_data}
    return dict(sorted(sum_lang_apperances.items(), key=lambda item: item[1], reverse=True))

def get_language_extent(language_data: list):
    lang_extent = {k: round(sum(x[1] for x in language_data if x[0] == k)/ len(repos), 4) for k, v in language_data}
    # prune languages with extent 0 due to rounding 
    pruned = {k: v for k, v in lang_extent.items() if v > 0}
    return dict(sorted(pruned.items(), key=lambda item: item[1], reverse=True))


In [ ]:
# start 
all_languages_data = get_repo_language_data(repos=repos)
# print(f'all languages: {sorted(all_languages, key=lambda d: d[1])}')
sum_lang_apperances = get_language_apperance(language_data=all_languages_data)
print(f'lang apperances: \n {sum_lang_apperances}')
lang_extent = get_language_extent(language_data=all_languages_data)
# lang_extent_sorted = dict(sorted(lang_extent.items(), key=lambda item: item[1], reverse=True))
print(f'lang extent: \n {lang_extent}')

# plot pie chart
# prune a bit more
langs_filtered = {k: v for k, v in lang_extent.items() if v >= 0.01}
print(langs_filtered)
if len(langs_filtered) != len(lang_extent):
    sum_langs_filtered = sum(langs_filtered.values())
    langs_filtered['Other'] = 1.0 - sum_langs_filtered

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = langs_filtered.keys()
sizes = langs_filtered.values()

fig1, ax1 = plt.subplots(figsize=(8, 8), dpi=200)
ax1.pie(sizes, labels=labels, pctdistance=0.8,  autopct='%1.2f%%', startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
plt.figure(figsize=(8, 8), dpi=200)
quadrant_chart(
    x=sum_lang_apperances.values(),
    y=lang_extent.values(),
    xtick_labels=['Low', 'High'],
    ytick_labels=['Low', 'High'],
    data_labels=sum_lang_apperances.keys(),
    highlight_quadrants=[1,2,4]
)
plt.title('Used languages in dPhoenix dependencies', fontsize=16)
plt.ylabel('Extent of Usage', fontsize=14)
plt.xlabel('Occurence of Language', fontsize=14)

In [ ]:
def get_repos_pr_data(repos: list, overwrite_existing_data = False) -> list:

    # prepare for data export
    file_name_repo_pr_data = "all_repo_pr_data.pkl"
    existing_file_found = False
    try:
        f = open(file_name_repo_pr_data, "rb")
        existing_file_found = True
    except:
        existing_file_found = False
    if (existing_file_found and not overwrite_existing_data):
        f = open(file_name_repo_pr_data, "rb")
        return pickle.load(f)
    else:
        repo_data = []
        # set today for age calculations outside the for loop to not get distortions between the different repos
        today = datetime.now()
        for repo in repos:
            # get last 1000 prs
            all_prs = get_last_1000_pull_requests(repo=repo["identifier"])

            # get statistics for PRs in general
            closed_prs = list(filter(lambda d: d.get('state') == 'closed', all_prs))
            # only check closed for merged as open PRs can not be merged
            merged_prs, not_merged_prs = filter_by_merge_status(pull_requests=closed_prs)
            # share of closed PRs not being merged
            share_of_not_merged_regular_prs = round(len(not_merged_prs) / len(closed_prs), 4) * 100
            merge_durations_regular_prs = get_durations_for_merge(merged_prs)

            dependabot_prs = list(filter(lambda d: d.get('user').get('login') == 'dependabot[bot]', all_prs))
            if len(dependabot_prs) != 0:
                open_dependabot_prs = list(filter(lambda d: d.get('state') == 'open', dependabot_prs))
                age_deltas = pd.DataFrame([(today - datetime.strptime(open_pr.get('created_at'), "%Y-%m-%dT%H:%M:%SZ")) for open_pr in open_dependabot_prs])
                closed_dependabot_prs = list(filter(lambda d: d.get('state') == 'closed', dependabot_prs))
                # only check closed for merged as open PRs can not be merged
                merged_dependabot_prs, not_merged_dependabot_prs = filter_by_merge_status(pull_requests=closed_dependabot_prs)
                # share of closed PRs not being merged
                share_of_not_merged_dependabot_prs = round(len(not_merged_dependabot_prs) / len(closed_dependabot_prs), 4) * 100

                # calculate mean age of merged
                if (len(merged_dependabot_prs) > 0):
                    merge_durations_dependabot_prs = get_durations_for_merge(merged_dependabot_prs)

                repo_data.append({
                    'repo_name': repo['identifier'],
                    'general': {
                        'raw_data_prs': all_prs,
                        'closed_pull_requests': closed_prs,
                        'merged_pull_requests': merged_prs,
                        'not_merged_pull_requests': not_merged_prs,
                        'share_closed_not_merged_pull_requests': share_of_not_merged_regular_prs,
                        'merge_durations': merge_durations_regular_prs if len(merged_prs) > 0 else None
                    },
                    'dependabot': {
                        'raw_data_prs': dependabot_prs,
                        'open_pull_requests': {
                            'raw_data_prs': open_dependabot_prs,
                            'age_deltas': age_deltas,
                        },
                        'closed_pull_requests': {
                            'raw_data_prs': closed_dependabot_prs,
                            'merged_pull_requests': merged_dependabot_prs,
                            'not_merged_pull_requests': not_merged_dependabot_prs,
                            'share_closed_not_merged_pull_requests': share_of_not_merged_dependabot_prs,
                            'merge_durations': merge_durations_dependabot_prs if len(merged_dependabot_prs) > 0 else None
                        }
                    
                    }
                })
            else:
                repo_data.append({
                    'repo_name': repo['identifier'],
                    'general': {
                        'raw_data_prs': all_prs,
                        'closed_pull_requests': closed_prs,
                        'merged_pull_requests': merged_prs,
                        'not_merged_pull_requests': not_merged_prs,
                        'share_closed_not_merged_pull_requests': share_of_not_merged_regular_prs,
                        'merge_durations': merge_durations_regular_prs if len(merged_prs) > 0 else None
                    },
                    'dependabot': {
                    }
                })
        # dump data to file
        open_file = open(file_name_repo_pr_data, "wb")
        pickle.dump(repo_data, open_file)
        open_file.close()
        return repo_data

def print_repo_statistics(repo_data: list):
    for r in repo_data:
        # output styling
        print('________________________________')
        print(f'{r["repo_name"]}:')
        print(f'found PRs: {len(r["general"]["raw_data_prs"])}')
        print(f'closed PRs in general: {len(r["general"]["closed_pull_requests"])}')
        print(f'closed and merged PRs: {len(r["general"]["merged_pull_requests"])}')
        print(f'closed and not merged PRs: {len(r["general"]["not_merged_pull_requests"])}')
        print(f'share of closed but not merged: {r["general"]["share_closed_not_merged_pull_requests"]}%')
        print(f'time to merge of PR:')
        if r["general"]["merge_durations"]:
            print_timedelta_statistics(r["general"]["merge_durations"])
        print('____________')
        if len(r["dependabot"]) != 0:
            print(f'Dependabot PRs: {len(r["dependabot"]["raw_data_prs"])}')
            print(f'open PRs: {len(r["dependabot"]["open_pull_requests"]["raw_data_prs"])}')
            print('age of open dependabot PRs:')
            print_timedelta_statistics(r["dependabot"]["open_pull_requests"]["age_deltas"])
            print(f'closed PRs: {len(r["dependabot"]["closed_pull_requests"]["raw_data_prs"])}')
            print(f'closed and merged PRs: {len(r["dependabot"]["closed_pull_requests"]["merged_pull_requests"])}')
            print(f'closed and not merged PRs: {len(r["dependabot"]["closed_pull_requests"]["not_merged_pull_requests"])}')
            print(f'share of closed but not merged: {r["dependabot"]["closed_pull_requests"]["share_closed_not_merged_pull_requests"]}%')
            if (len(r["dependabot"]["closed_pull_requests"]["merged_pull_requests"]) > 0):
                print(f'time to merge of dependabot PR:')
                print_timedelta_statistics(r["dependabot"]["closed_pull_requests"]["merge_durations"])
        else: 
            print(f'no dependabot usage in history of last 1000 PRs')


def get_durations_for_merge(pull_requests: list):
    merge_deltas = [(datetime.strptime(pr.get('merged_at'), "%Y-%m-%dT%H:%M:%SZ") - datetime.strptime(pr.get('created_at'), "%Y-%m-%dT%H:%M:%SZ")) for pr in pull_requests]
    return merge_deltas
    
def print_timedelta_statistics(time_deltas: list):
    time_deltas = pd.DataFrame(time_deltas)
    print(f'mean: {np.mean(time_deltas)}')
    print(f'std: {np.std(time_deltas)}')
    print(f'min: {np.min(time_deltas)}')
    print(f'max: {np.max(time_deltas)}')
    
def get_last_1000_pull_requests(repo: str):
    pull_requests = []
    response_status = 200
    page_index = 1
    while (response_status == 200 and page_index <= 10):
        res = requests.get(f"{base_repos_url}{repo}/pulls" , headers=headers, params={
            'per_page':100, 'page':page_index, 'state': 'all'
        })
        pull_requests += res.json()
        response_status = res.status_code
        page_index += 1
    return pull_requests

def get_last_1000_commits(repo: str):
    commits = []
    response_status = 200
    page_index = 1
    while (response_status == 200 and page_index <= 10):
        res = requests.get(f"{base_repos_url}{repo}/commits" , headers=headers, params={
            'per_page':100, 'page':page_index,
        })
        commits += res.json()
        response_status = res.status_code
        page_index += 1
    return commits

def filter_by_merge_status(pull_requests: list):
    merged_dependabot_prs = []
    not_merged_dependabot_prs = []
    for pr in pull_requests:
        if pr.get('merged_at') is not None:
            merged_dependabot_prs.append(pr)
        else:
            not_merged_dependabot_prs.append(pr)
    return merged_dependabot_prs, not_merged_dependabot_prs

def plot_boxplot_merge_durations(time_deltas: list, dependency_names: list, fig_title: str, manual_limits: tuple):
    deltas_in_days = [[ delta.total_seconds()/(60*60*24) for delta in deltas] for deltas in time_deltas]
    fig = plt.figure(figsize =(2.5*len(dependency_names), 6), dpi=200)
    ax = fig.add_subplot(111)
    
    # Creating plot
    plt.boxplot(deltas_in_days)
    # x-axis labels
    ax.set_xticklabels(f'{dependency_name.split("/")[0]}/\n{dependency_name.split("/")[1]}\n(n= {len(time_deltas[ind])})' for ind, dependency_name in enumerate(dependency_names) )
    # Adding title
    plt.title(fig_title)
    if (manual_limits):
        plt.ylim(manual_limits)
    # show plot
    plt.show()

def plot_violinplot_merge_durations(time_deltas: list, dependency_names: list, fig_title: str, manual_limits: tuple):
    deltas_in_days = [[ delta.total_seconds()/(60*60*24) for delta in deltas] for deltas in time_deltas]
    fig = plt.figure(figsize =(2.5*len(dependency_names), 6), dpi=200)
    ax = fig.add_subplot(111)
    
    # Creating plot
    plt.violinplot(deltas_in_days)
    # sns.violinplot(deltas_in_days)
    # x-axis labels
    ax.set_xticklabels(f'{dependency_name.split("/")[0]}/\n{dependency_name.split("/")[1]}\n(n= {len(time_deltas[ind])})' for ind, dependency_name in enumerate(dependency_names) )
    # Adding title
    plt.title(fig_title)
    if (manual_limits):
        plt.ylim(manual_limits)
    # show plot
    plt.show()

In [ ]:
# start
repo_data = get_repos_pr_data(repos=repos)
print_repo_statistics(repo_data=repo_data)

In [ ]:
# plot general pr times
plot_boxplot_merge_durations([repo['general']['merge_durations'] for repo in repo_data[:5] if repo['general']['merge_durations'] != None], [repo['repo_name'] for repo in repo_data[:5] if repo['general']['merge_durations'] != None], fig_title="Days unitl opened pull requests get merged", manual_limits=(-5, 180))
# plit up in two figs
plot_boxplot_merge_durations([repo['general']['merge_durations'] for repo in repo_data[5:] if repo['general']['merge_durations'] != None], [repo['repo_name'] for repo in repo_data[5:] if repo['general']['merge_durations'] != None], fig_title="Days unitl opened pull requests get merged", manual_limits=(-5, 180))
# plot general pr times
plot_violinplot_merge_durations([repo['general']['merge_durations'] for repo in repo_data[:5] if repo['general']['merge_durations'] != None], [repo['repo_name'] for repo in repo_data[:5] if repo['general']['merge_durations'] != None], fig_title="Days unitl opened pull requests get merged", manual_limits=(-5, 180))
# plit up in two figs
plot_violinplot_merge_durations([repo['general']['merge_durations'] for repo in repo_data[5:] if repo['general']['merge_durations'] != None], [repo['repo_name'] for repo in repo_data[5:] if repo['general']['merge_durations'] != None], fig_title="Days unitl opened pull requests get merged", manual_limits=(-5, 180))

# plot dependabot times
plot_boxplot_merge_durations([repo['dependabot']['closed_pull_requests']['merge_durations'] for repo in repo_data if repo['dependabot'] != {} and repo['dependabot']['closed_pull_requests']['merge_durations'] != None], [repo['repo_name'] for repo in repo_data if repo['dependabot'] != {}and repo['dependabot']['closed_pull_requests']['merge_durations'] != None], fig_title="Days until opened Dependabot pull requests get merged", manual_limits=None)

In [ ]:
mean_share_not_merged = [repo['general']['share_closed_not_merged_pull_requests'] for repo in repo_data[1:]]
print(mean_share_not_merged)
print(f'mean: {np.mean(mean_share_not_merged)}')
print(f'std: {np.std(mean_share_not_merged)}')
print(f'min: {np.min(mean_share_not_merged)}')
print(f'max: {np.max(mean_share_not_merged)}')

In [ ]:
mean_share_not_merged = [repo['dependabot']['closed_pull_requests']['share_closed_not_merged_pull_requests'] for repo in repo_data[1:] if repo['dependabot'] != {}]
print(mean_share_not_merged)
print(f'mean: {np.mean(mean_share_not_merged)}')
print(f'std: {np.std(mean_share_not_merged)}')
print(f'min: {np.min(mean_share_not_merged)}')
print(f'max: {np.max(mean_share_not_merged)}')

In [ ]:
mean_avrg_merge_time = [np.mean(repo['general']['merge_durations']) for repo in repo_data[1:]]
print(f'mean: {np.mean(mean_avrg_merge_time)}')
# print(f'std: {np.std(mean_avrg_merge_time)}')
print(f'min: {np.min(mean_avrg_merge_time)}')
print(f'max: {np.max(mean_avrg_merge_time)}')

In [ ]:
def refresh_debricked_access_token():
    # refresh access token
    refresh_url = f'https://debricked.com/api/login_refresh?refresh_token={debricked_access_token}'
    res = requests.post(refresh_url)
    data = res.json()
    headers_debricked = {}
    headers_debricked['Authorization'] = f"Bearer {data['token']}"
    return headers_debricked
    
def get_repo_dependencies(repo_id):
     dependencies = []
     page_index = 1
     data_page = None
     while (data_page != []):
          res = requests.get(debricked_api_url + '/open/dependencies/get-dependencies',
          headers=headers_debricked, params={
               'repositoryId': repo_id,
               'sortColumn': 'name',
               'order': 'desc',
               'rowsPerPage': 50,
               'page': page_index
          })
          data = res.json()
          dependencies += data['dependencies']
          data_page = data['dependencies']
          page_index += 1
     print(dependencies)
     return dependencies

def get_repo_vulnerabilities(repo_id):
   vulnerabilities = []
   page_index = 1
   data_page = None
   while (data_page != []):
      res = requests.get(debricked_api_url + '/open/vulnerabilities/get-vulnerabilities',
      headers=headers_debricked, params={
         'repositoryId': repo_id,
         'sortColumn': 'name',
         'order': 'desc',
         'rowsPerPage': 50,
         'page': page_index
      })
      data = res.json()
      vulnerabilities += data['vulnerabilities']
      data_page = data['vulnerabilities']
      page_index += 1
   return vulnerabilities

def get_vulnerabilities_with_fix(vulnerabilities: list):
     vulns_with_fix = list(filter(lambda vuln: vuln['fixesAndExploits'][0]['type'] == 'manual-fix', vulnerabilities))
     return vulns_with_fix

def plot_as_bar_chart(repo_data: list):

     labels = [f'{repo["repo_name"].split("/")[0]}/\n{repo["repo_name"].split("/")[1]}' for repo in repo_data]
     dependencies  = [repo['nmb_dependencies'] for repo in repo_data]
     vulnerabilities = [repo['nmb_vulnerabilities'] for repo in repo_data]
     vulnerabilities_with_fix = [repo['nmb_vulnerabilities_with_fix'] for repo in repo_data]

     x = np.arange(len(labels))  # the label locations
     width = 0.3  # the width of the bars

     fig = plt.figure(figsize =(2*len(labels), 6), dpi=200)
     ax = fig.add_subplot(111)
     rects1 = ax.bar(x - width, dependencies, width, label='Dependencies')
     rects2 = ax.bar(x , vulnerabilities, width, label='Vulnerabilities')
     rects3 = ax.bar(x + width, vulnerabilities_with_fix, width, label='Vulnerabilities having a fix')

     # Add some text for labels, title and custom x-axis tick labels, etc.
     ax.set_ylabel('Quantity')
     ax.set_title('Dependencies and vulnerabilties by repository')
     ax.set_xticks(x, labels)
     ax.legend()

     ax.bar_label(rects1, padding=3)
     ax.bar_label(rects2, padding=3)
     ax.bar_label(rects3, padding=3)

     fig.tight_layout()

     plt.show()


In [ ]:
# upload data to Debricked, you only need to run this command once!
headers_debricked = refresh_debricked_access_token()
for repo in repos:
    if repo['dependency_files'] != []:
        for dependency_file in repo['dependency_files']:
            upload_id = ''
            print(repo)
            print(dependency_file)
            res = requests.get(base_repos_url + f'{repo["identifier"]}/contents/{dependency_file}', headers=headers )
            data = res.json()
            res = requests.get(data['download_url'])
            file_content = res.content.decode('utf-8')
            if upload_id is not '':
                data = {
                    'repositoryName': repo["identifier"],
                    'commitName':'analysis',
                    'upload_id': upload_id
                } 
            else :
                data = {
                    'repositoryName': repo["identifier"],
                    'commitName':'analysis'
                } 
            url = 'https://debricked.com/api/1.0'
            res = requests.post(debricked_api_url + '/open/uploads/dependencies/files',
                files = {"fileData": file_content}, headers=headers_debricked, data=data )
            data = res.json()
            if upload_id is '':
                upload_id = data['ciUploadId']

        data = {
            'ciUploadId': upload_id
        }
        res = requests.post(debricked_api_url + '/open/finishes/dependencies/files/uploads',
            headers=headers_debricked, data=data )
        print(res)

In [ ]:
headers_debricked = refresh_debricked_access_token()
res = requests.get(debricked_api_url + '/open/repositories/get-repositories',
     headers=headers_debricked )
all_user_repos = res.json()['repositories']
repo_data_for_plotting = []
for repo in repos:
     if repo['dependency_files'] != []:
          print('_________________________')
          current_repo = list(filter(lambda r: r['name']['name'] == repo['identifier'], all_user_repos))
          current_repo_id = current_repo[0]['id']
          print(f'{current_repo[0]["name"]["name"]}:')
          nmb_dependencies = len(get_repo_dependencies(current_repo_id))
          print(f'dependencies: {nmb_dependencies}')
          vulnerabilities = get_repo_vulnerabilities(current_repo_id)
          nmb_vulnerabilities = len(vulnerabilities)
          nmb_vulnerabilities_with_fix = len(get_vulnerabilities_with_fix(vulnerabilities=vulnerabilities))
          print(f'vulnerabilities: {nmb_vulnerabilities}')
          print(f'from those having a fix: {nmb_vulnerabilities_with_fix}')

          repo_data_for_plotting.append({
               'repo_name': repo['identifier'],
               'nmb_dependencies': nmb_dependencies,
               'nmb_vulnerabilities': nmb_vulnerabilities,
               'nmb_vulnerabilities_with_fix': nmb_vulnerabilities_with_fix
          })
     else:
          print('_________________________')
          print(f'{repo["identifier"]}:')
          print(f'no dependency files added!')


In [ ]:
plot_as_bar_chart(repo_data=repo_data_for_plotting[1:])
sum_dependencies = sum(repo['nmb_dependencies'] for repo in repo_data_for_plotting[1:])
print(sum_dependencies)
shares_dependencies_vulnerabilities= [repo['nmb_vulnerabilities']/repo['nmb_dependencies'] for repo in repo_data_for_plotting[1:]]
print(f'shares_dependencies_vulnerabilities:')
print(f'mean: {round(np.mean(shares_dependencies_vulnerabilities)*100,2)}%')
print(f'std: {round(np.std(shares_dependencies_vulnerabilities)*100,2)}%')
print(f'min: {round(np.min(shares_dependencies_vulnerabilities)*100,2)}%')
print(f'max: {round(np.max(shares_dependencies_vulnerabilities)*100,2)}%')

